In [1]:
# import packages
from joblib import dump, load
import numpy as np
import pandas as pd
import timeit
import time
from sklearn import preprocessing
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as pkl
from umap.umap_ import UMAP
import umap.plot

from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier

/Users/melloo21/opt/anaconda3/envs/cs4220_pathogen_detection/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/melloo21/opt/anaconda3/envs/cs4220_pathogen_detection/lib/python3.8/site-packages/umap/plot.py:203: NumbaDeprecationWarning: The keyword argument 'nopython=False' was supplied. From Numba 0.59.0 the default is being changed to True and use of 'nopython=False' will raise a warning as the argument will have no effect. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit(nopython=False)


## Parsing Input Data

First, some dataset statistics. We load our training labels (ground truth) and see how many reads a category (species) have.

In [2]:
label_df = pd.read_csv('./training_data/train_labels.csv')
label_df['species_name'].value_counts()

species_name
decoy                              413476
burkholderia_pseudomallei            3533
pseudomonas_aeruginosa               3126
klebsiella_michiganensis             2989
mycobacterium_ulcerans               2910
klebsiella_pneumoniae                2806
serratia_liquefaciens                2629
vibrio_parahaemolyticus              2579
salmonella_enterica_typhimurium      2507
yersinia_enterocolitica              2276
stenotrophomonas_maltophilia         2217
mycobacterium_tuberculosis           2175
clostridioides_difficile             2007
acinetobacter_baumannii              1964
legionella_pneumophila               1753
listeria_monocytogenes               1479
staphylococcus_aureus                1384
staphylococcus_pseudintermedius      1328
corynebacterium_ulcerans             1266
corynebacterium_diphtheriae          1194
streptococcus_suis                   1092
neisseria_gonorrhoeae                1087
streptococcus_agalactiae             1060
streptococcus_pneumon

It seems that we have a lot of decoy reads (decoy means sequencing reads from human or commensal species).

In [3]:
# snippet to load the grouth truth training labels and normalize the label predictions.
# your trained model will predict in this space (26 classes - pathogens and decoy)

le = preprocessing.LabelEncoder()
le.fit(label_df['species_name'].unique())
y_index = le.transform(label_df['species_name'].values)
label_df['labels'] = y_index

In this project, we try to use canonical $k$-mer profiles to represent each read in the input. Here, we are using $k=6$ and consequently 2081 features (including 1 feature `IGNORE` for ambiguous $k$-mer) for each read. Read [this reference](https://bioinfologics.github.io/post/2018/09/17/k-mer-counting-part-i-introduction/) for more information.

To help you save time, we implemented a utility class `CS4220Dataset` that can
- take in raw reads as input (`.fasta`, `.fa` files), and turn them into $k$-mer profiles, or
- take in $k$-mer profile as input (`.npy` files),
- allow you to sample data or create $k$-mer profile on the fly (during training) to save memory.

In [4]:
# Load dictionary that maps k-mer to their corresponding index.
# A k-mer and its reverse complement are mapped to the same index.

import json

with open("./training_data/6-mers.json", 'r') as dict_file:
    canonical_kmer_dict = json.load(dict_file)

In [5]:
# We define a utility function here that turns sequences to their 6-mer profiles.

def sequence_to_kmer_profile(sequence : str, k : int = 6):
    """
    Return the k-mer profile of the input sequence (string)
    """
    res = np.zeros(len(set(canonical_kmer_dict.values())))
    for i in range(len(sequence) - k + 1):
        k_mer = sequence[i:i + k]
        if k_mer in canonical_kmer_dict:
            res[canonical_kmer_dict[k_mer]] += 1
        else:
            res[-1] += 1

    res /= np.sum(res)
    return res

In [6]:
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader

class CS4220Dataset(Dataset):
    def __init__(self, data_file, label_df=None, k=6, samples_index=None, kmer_profile_on_the_fly=False, dtype=np.float32):
        """
        Dataset class to load large CS4220 sequence database.

        Args:
            - data_file (`str`): Can either be a *.fasta file if the input is raw reads, or *.npy file
                                 if the input is k-mer profile.
            - label_df (`pd.DataFrame` or `None`): A dataframe with "labels" column indicating the label
                                                   of the data (must match with data_file), or `None` if there is
                                                   no label (in the case of test sets).
            - k (`int`): The lengt of k-mer. We use 6 in this project.
            - samples_index (`List` or `None`): list of indices of data we sample from the data file. You
                                                can use this if the dataset is very large and can't fit in memory.
                                                set this to `None` if you want to use all the data.
            - kmer_profile_on_the_fly (`bool`): If input data_file is raw reads and this set to `True`,
                                                we will build k-mer profile on the fly. This is helpful if you want to
                                                alter the input sequences during training, or the k-mer profile can't fit in memory.
                                                Otherwise, we build k-mer profile in advance, which will speed up the
                                                training process.
            - dtype: type to store the k-mer profile. You may use, for example, `np.float32` for better precision,
                     or `np.float16` for smaller memory usage. If loaded from ".npy" file, it is always `np.float16`.
        """
        self.data_file = data_file

        if ".fasta" in data_file or ".fa" in data_file or ".fna" in data_file:
            self.is_raw_reads = True
        elif ".npy" in data_file:
            self.is_raw_reads = False
        else:
            raise TypeError(f"The input file must be either a fasta file containing raw reads (.fasta, .fa, .fna) or a numpy file containing k-mer profiles (.npy).")


        self.label_df = label_df
        self.kmer_profile_otf = kmer_profile_on_the_fly

        # k-mer length, set to be 6.
        self.k = k

        # the samples we take from the read dataset
        self.samples_index = samples_index

        self.dtype = dtype

        # Load the data and store in self.reads and self.labels
        self.X = []
        self.Y = []
        self._read_labels()
        self._read_data()


    def _read_labels(self):
        """
        Read the labels and record them in self.labels.
        """
        if self.label_df is None:
            self.Y = None
        elif self.samples_index is None:
            # Load the whole dataset
            self.Y = list(self.label_df["labels"])
        else:
            # Load only the data corresponding to the sampled index
            self.Y = list(self.label_df.iloc[self.samples_index]["labels"])


    def _read_data(self):
        if self.is_raw_reads:
            # Read the fasta file
            with open(self.data_file, 'r') as fasta_file:
                lines = fasta_file.readlines()

            read_range = self.samples_index if self.samples_index is not None else range(int(len(lines) / 2))
            if not self.kmer_profile_otf:
                self.X = np.zeros(
                    (len(read_range), len(set(canonical_kmer_dict.values()))),
                    dtype=self.dtype
                )

            for i, j in enumerate(tqdm(read_range, desc=f"Parsing fasta file {self.data_file}")):
                read = lines[j * 2 + 1].strip()
                if self.kmer_profile_otf:
                    # If chose to do k-mer profiling on the fly, simply store the reads
                    self.X.append(read)
                else:
                    # Otherwise, do k-mer profiling during training/testing, cost more time during training/testing
                    self.X[i, :] = sequence_to_kmer_profile(read, self.k)
        else:
            # Read the .npy file, and load the numpy matrix
            # Each row corresponds to a read, and each column corresponds to a k-mer (see training_data/6-mers.txt).
            self.X = np.load(self.data_file)
            if self.samples_index is not None:
                self.X = self.X[self.samples_index, :]

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        """
        If you are using pytorch, this function helps taking data points during each epoch
        of your training.
        """
        x = self.X[idx]
        if self.kmer_profile_otf:
            read_tensor = torch.tensor(sequence_to_kmer_profile(x, self.k), dtype=self.dtype)
        else:
            read_tensor = torch.tensor(x)

        label = self.Y[idx] if self.Y is not None else None
        return read_tensor, label


## Quick Exploratory Data Analysis

We can use a [UMAP plot](https://www.scdiscoveries.com/blog/knowledge/what-is-a-umap-plot/) to try to group reads in the same species together, according to their $k$-mer profiles.

What interesting patterns can you notice? Some observations might be:
- Some species are pretty separated in clusters.
- Even though most species lie in separate groups, we can see some regions where points from different species are overlapping.
- There are some "lost points" for pretty much every species.

Do you think these pieces of information can help you in something? Think about it.

## Training

OK, time for training. As baseline we will choose the simplest [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression) model.

You can fine-tune the logistic regression hyperparameters such as the penalty and regularization term.

```python
from sklearn.model_selection import GridSearchCV

#list of items to tune
parameters_lr = [{'penalty': ['l1','l2'], 'C': [0.001, 0.01, 0.1, 1, 10, 100]}]

starting_time = timeit.default_timer()

regr = LogisticRegression(random_state=2023, solver='saga', n_jobs=-1, class_weight='balanced', max_iter=50, verbose=1)

grid_search_lr = GridSearchCV(estimator = regr,
                              param_grid = parameters_lr,
                              scoring = 'accuracy',
                              cv = 3,
                              n_jobs = -1)

# the last column of our 6mer training dataset can be ignored (training labels)
grid_search_lr.fit(sampled_dataset.X, sampled_dataset.Y)
best_accuracy_lr = grid_search_lr.best_score_
best_parameter_lr = grid_search_lr.best_params_

```

This might take a long time. Here, we skip this step and jump straight to training.

In [7]:
# samples_index = label_df.groupby('labels').sample(100, replace=True).index
# threshold=0.7
# input_file_path = './training_data/train_6mers.npy'
# dataset = CS4220Dataset(input_file_path, label_df, samples_index=samples_index)
# data = dataset.X
# labels = dataset.Y

# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)
# model = SVC(kernel='rbf', C=100.0, probability=True)
# ovr = OneVsRestClassifier(model)
# ovr.fit(X_train_scaled, y_train)
# pred_prob = ovr.predict_proba(X_test_scaled)

# final_predictions = le.inverse_transform(
#                         np.unique([np.argmax(item) for item in pred_prob if len(np.where(item >= threshold)[0]) >= 1]
#                     ))
# final_predictions

### Check on sampled data

In [8]:
# samples_index = label_df.groupby('labels').sample(800, replace=False).index

# input_file_path = './training_data/train_6mers.npy'
# dataset = CS4220Dataset(input_file_path, label_df, samples_index=samples_index)
# data = dataset.X
# labels = dataset.Y

# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# num_classes = len(set(labels))
# threshold = 0.9

# # Define the parameter grid
# param_grid = {'estimator__C': [0.1, 1, 10, 100], 'estimator__kernel': ['linear', 'rbf', 'poly']}

# # Initialize SVM classifier
# model = OneVsRestClassifier(SVC(probability=True))

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(model, param_grid, cv=5, verbose=2)
# search_result = grid_search.fit(X_train_scaled, y_train)

# # Print the best parameters and the corresponding accuracy
# print("Best Parameters:", grid_search.best_params_)
# print("Best Accuracy:", grid_search.best_score_)


In [9]:
# means = search_result.cv_results_['mean_test_score']
# params = search_result.cv_results_['params']

# for mean, param in zip(means, params):
#     print("%f with: %r" % (mean, param))


# # Store the parameters of the best model
# best_params = model.best_params_

In [10]:
# best_kernel = 'rbf'
# best_C = 10
# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# threshold = 0.7
# model = SVC(kernel=best_kernel, C=best_c)
# model.fit(X_train_scaled, y_train)

# pred = model.predict(X_test_scaled)
# pred_prob = model.predict_proba(X_test_scaled)
# final_predictions = le.inverse_transform(
#                         np.unique([np.argmax(item) for item in pred_prob if len(np.where(item >= threshold)[0]) >= 1]
#                     ))

In [11]:
# split into train and test
# samples_index = label_df.groupby('labels').sample(800, replace=True).index

# input_file_path = './training_data/train_6mers.npy'
# dataset = CS4220Dataset(input_file_path, label_df, samples_index=samples_index)
# data = dataset.X
# labels = dataset.Y

# best_kernel = 'rbf'
# best_C = 100
# X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# model = SVC(kernel=best_kernel, C=best_C)
# model.fit(X_train_scaled, y_train)
# pred_prob = ovr.predict_proba(X_test_scaled)

In [12]:
# run on all train
samples_index = label_df.groupby('labels').sample(800, replace=True).index

input_file_path = './training_data/train_6mers.npy'
dataset = CS4220Dataset(input_file_path, label_df, samples_index=samples_index)
data = dataset.X
labels = dataset.Y

best_kernel = 'rbf'
best_C = 1

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(data)

model = OneVsRestClassifier(SVC(kernel=best_kernel, C=best_C, probability=True))
model.fit(data, labels)

OneVsRestClassifier(estimator=SVC(C=1, probability=True))

In [9]:
from joblib import dump, load
# dump(model, 'models/svc_noscaler_c1.joblib')
# model = load('models/svc_noscaler.joblib')

In [12]:
def jaccard_index_per_patient(patient_id, preds):
    df_true = pd.read_csv('test_data/patient{}_labels.csv'.format(patient_id))
    tp, fp, tp_fn = 0, 0, df_true['labels'].shape[0]
    print('my predition(s) for patient {}:'.format(patient_id))
    print(preds)
    print('true pathogen')
    print(df_true['labels'].values)
    # if don't predict any pathogen, it means there is only decoy in the test dataset (your prediction)
    if len(preds) == 0:
        preds = ['decoy']
    for item in np.unique(preds):
        if item in df_true['labels'].values:
            tp += 1
        else:
            fp += 1
    #you have to predict all labels correctly, but you are penalized for any false positive
    return tp / (tp_fn + fp)

In [7]:
#prediction for all patients

threshold = 0.99

all_jaccard_index = []
for patient_id in range(10):
    print('predicting for patient {}'.format(patient_id))

    with open('test_data/patient{}_6mers.npy'.format(patient_id), 'rb') as read_file:
        df_test = np.load(read_file)

    # regr.predict relies on argmax, thus predict to every single read and you will end up with many false positives
    # y_pred = model.predict(df_test)

    # we can use regr.predict_proba to find a good threshold and predict only for case where the model is confident.
    # here I apply 0.95 as the cutoff for my predictions, let's see how well my model will behave...
    y_predprob = model.predict_proba(df_test)

    # we get only predictions larger than the threshold and if there is more than one, we take the argmax again
    final_predictions = le.inverse_transform(
                            np.unique([np.argmax(item) for item in y_predprob if len(np.where(item >= threshold)[0]) >= 1]
                        ))

    # my pathogens dectected, decoy will be ignored
    final_predictions = [item for item in final_predictions if item !='decoy']

    ji = jaccard_index_per_patient(patient_id, final_predictions)
    print('Jaccard index: {}'.format(ji))
    all_jaccard_index.append(ji)

predicting for patient 0


NameError: name 'model' is not defined

In [16]:
print(['patient {}: {}'.format(c,item) for c, item in enumerate(all_jaccard_index)], 'avg: {}'.format(np.mean(all_jaccard_index)))

['patient 0: 0.125', 'patient 1: 0.09090909090909091', 'patient 2: 0.5', 'patient 3: 0.3333333333333333', 'patient 4: 0.08333333333333333', 'patient 5: 0.125', 'patient 6: 0.07692307692307693', 'patient 7: 0.2', 'patient 8: 0.2', 'patient 9: 0.1'] avg: 0.18344988344988344


In [13]:
#prediction for all patients
model = load('models/svc_noscaler_c1.joblib')
threshold = 0.99

all_jaccard_index = []
for patient_id in range(10):
    print('predicting for patient {}'.format(patient_id))

    with open('test_data/patient{}_6mers.npy'.format(patient_id), 'rb') as read_file:
        df_test = np.load(read_file)

    # regr.predict relies on argmax, thus predict to every single read and you will end up with many false positives
    # y_pred = model.predict(df_test)

    # we can use regr.predict_proba to find a good threshold and predict only for case where the model is confident.
    # here I apply 0.95 as the cutoff for my predictions, let's see how well my model will behave...
    y_predprob = model.predict_proba(df_test)

    # we get only predictions larger than the threshold and if there is more than one, we take the argmax again
    final_predictions = le.inverse_transform(
                            np.unique([np.argmax(item) for item in y_predprob if len(np.where(item >= threshold)[0]) >= 1]
                        ))

    # my pathogens dectected, decoy will be ignored
    final_predictions = [item for item in final_predictions if item !='decoy']

    ji = jaccard_index_per_patient(patient_id, final_predictions)
    print('Jaccard index: {}'.format(ji))
    all_jaccard_index.append(ji)

/Users/melloo21/opt/anaconda3/envs/cs4220_pathogen_detection/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator SVC from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/melloo21/opt/anaconda3/envs/cs4220_pathogen_detection/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator LabelBinarizer from version 1.2.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/melloo21/opt/anaconda3/envs/cs4220_pathogen_detection/lib/python3.8/site-packages/sklearn/base.py:348: InconsistentVersi

predicting for patient 0
my predition(s) for patient 0:
['staphylococcus_aureus']
true pathogen
['staphylococcus_aureus']
Jaccard index: 1.0
predicting for patient 1
my predition(s) for patient 1:
['neisseria_gonorrhoeae', 'salmonella_enterica_typhimurium']
true pathogen
['staphylococcus_pyogenes']
Jaccard index: 0.0
predicting for patient 2
my predition(s) for patient 2:
['burkholderia_pseudomallei', 'corynebacterium_ulcerans']
true pathogen
['burkholderia_pseudomallei' 'corynebacterium_ulcerans']
Jaccard index: 1.0
predicting for patient 3
my predition(s) for patient 3:
['pseudomonas_aeruginosa']
true pathogen
['pseudomonas_aeruginosa']
Jaccard index: 1.0
predicting for patient 4
my predition(s) for patient 4:
['corynebacterium_diphtheriae']
true pathogen
['corynebacterium_diphtheriae']
Jaccard index: 1.0
predicting for patient 5
my predition(s) for patient 5:
['streptococcus_pneumoniae']
true pathogen
['streptococcus_pneumoniae']
Jaccard index: 1.0
predicting for patient 6
my predit

In [14]:
print(['patient {}: {}'.format(c,item) for c, item in enumerate(all_jaccard_index)], 'avg: {}'.format(np.mean(all_jaccard_index)))

['patient 0: 1.0', 'patient 1: 0.0', 'patient 2: 1.0', 'patient 3: 1.0', 'patient 4: 1.0', 'patient 5: 1.0', 'patient 6: 1.0', 'patient 7: 0.3333333333333333', 'patient 8: 1.0', 'patient 9: 0.5'] avg: 0.7833333333333333


In [15]:
final_predictions

['burkholderia_pseudomallei', 'neisseria_gonorrhoeae']

We can now save our trained model for later usage. This is an example of how you can send your model to our final evaluation.

Then the model can be loaded using

```python
# load trained model
regr = load('models/baseline.joblib')
```

## Evaluation of Model

Now that you have your trained model, you can use it on each of the patient's read dataset and try to find the pathogens that appear in each patient.

Your model is evaluated based on [**Jaccard index**](https://en.wikipedia.org/wiki/Jaccard_index#Jaccard_index_in_binary_classification_confusion_matrices). For patient $i$, let $P$ be the set of pathogen species your model predicted (or $\{\text{decoy}\}$ if there is no pathogens predicted), and $T$ the set of pathogen species the patient actually have (or $\{\text{decoy}\}$ if there is no pathogens in the reads), the score for your model is

$$\text{Jaccard index}=\frac{|P\cap T|}{|P \cup T|}$$

Going back to our model: since we are using logistic regression, our model will classify each read to the class that has the highest probability of having the read. If we report all the species that a read has been classified to, then we may end up with a lot of false positives (why?).

One potential way to counter this is to define a threshold. Here I used 0.95: I only report a species if I am 95% confident that one read comes from that species. Let's see how well my model will behave...

So the overall score for my model is 0.45. Not a bad start, but still much room for improvement. You don't necessarily need to work on this baseline; this was just released as an example. Have fun!!!